# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath="../output_data/cities.csv"
cities_df=pd.read_csv(filepath)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cabo San Lucas,22.89,-109.91,86.00,65,5,3.36,MX,1602864675
1,Dedza,-14.38,34.33,69.71,55,63,3.71,MW,1602864675
2,Qingdao,36.10,120.37,54.00,93,0,2.24,CN,1602864675
3,Lavrentiya,65.58,-171.00,39.00,93,100,30.80,RU,1602864675
4,Yangcun,39.36,117.06,55.40,39,0,4.47,CN,1602864675


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=cities_df[['Lat','Lng']]
weights=cities_df['Humidity']
# mid_map is the Lat and Lng coordinates so I could center the map on the screen.
mid_map=(0,15)
fig=gmaps.figure(center=mid_map,zoom_level=1)
heat_layer=gmaps.heatmap_layer(locations,weights=weights)
heat_layer.max_intensity=100
heat_layer.point_radius=5
heat_layer.dissipation=False
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# I included humidity level as well as the temp, cloudiness and windspeed.
hitemp=80
lwtemp=70
humidity=60
cloudiness=25
windspeed=10
my_spots_df=cities_df.loc[
                    (cities_df['Max Temp']>=lwtemp) & (cities_df['Max Temp']<=hitemp) &
                    (cities_df['Humidity']<=humidity) &
                    (cities_df['Cloudiness']<=cloudiness) & 
                    (cities_df['Wind Speed']<=windspeed) 
                    ,:]
my_spots_df              

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,San Quintín,30.48,-115.95,77.67,46,7,6.78,MX,1602864676
45,Alice Springs,-23.70,133.88,75.00,33,0,3.36,AU,1602864563
57,Atascadero,35.49,-120.67,75.99,57,1,4.56,US,1602864492
59,Casa Grande,32.88,-111.76,73.40,26,1,7.29,US,1602864683
61,Castro,-24.79,-50.01,72.28,52,1,8.01,BR,1602864683
67,Cape Town,-33.93,18.42,78.01,29,0,8.05,ZA,1602864632
133,Saldanha,-33.01,17.94,75.20,36,0,9.17,ZA,1602864691
373,Hīt,33.64,42.83,79.84,21,0,6.64,IQ,1602864717
381,Abu Samrah,35.30,37.18,71.08,42,0,7.76,SY,1602864717
390,Nizwá,22.93,57.53,77.11,27,0,8.70,OM,1602864718


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#Limit lookups to 10 vacation spots.
lookup_limit=10
hotel_df=my_spots_df.copy()
hotel_df['Hotel Name']=''

params={
        "query":'hotel',
        "radius":5000,
        "type":"lodging",
        "language":"en",
        "key":g_key
}

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
x=0
for index, row in hotel_df.iterrows():
    if (x==lookup_limit):
        break
    x+=1
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    hotel_names=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_names["results"][0]["name"]
    except:
        hotel_df.loc[index,"Hotel Name"]="No Hotel"
hotel_df.head(lookup_limit)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,San Quintín,30.48,-115.95,77.67,46,7,6.78,MX,1602864676,Old Mill Hotel
45,Alice Springs,-23.70,133.88,75.00,33,0,3.36,AU,1602864563,Desert Palms Alice Springs
57,Atascadero,35.49,-120.67,75.99,57,1,4.56,US,1602864492,Holiday Inn Express Hotel & Suites Atascadero
59,Casa Grande,32.88,-111.76,73.40,26,1,7.29,US,1602864683,Holiday Inn Casa Grande
61,Castro,-24.79,-50.01,72.28,52,1,8.01,BR,1602864683,CHACARA BAILLY
67,Cape Town,-33.93,18.42,78.01,29,0,8.05,ZA,1602864632,Southern Sun Waterfront Cape Town
133,Saldanha,-33.01,17.94,75.20,36,0,9.17,ZA,1602864691,Blue Bay Lodge
373,Hīt,33.64,42.83,79.84,21,0,6.64,IQ,1602864717,Aboody Alhachem
381,Abu Samrah,35.30,37.18,71.08,42,0,7.76,SY,1602864717,No Hotel
390,Nizwá,22.93,57.53,77.11,27,0,8.70,OM,1602864718,Orient Apartments


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))